In [2]:
import praw
import json
from datetime import datetime

reddit = praw.Reddit(
    client_id='B-qrtXmBJd2BgAAwmKozQg',
    client_secret='iVwh4ZpT35sQMBvye7nKaz8839LDPQ',
    user_agent='YOUR_USER_AGENT'
)

subreddit_name = 'IndianSkincareAddicts'
subreddit = reddit.subreddit(subreddit_name)
search_query = 'Love Beauty and Planet'
search_results = subreddit.search(search_query, limit=200)

parsed_posts = []
for post in search_results:
    # Convert post creation time to human-readable format
    created_date = datetime.utcfromtimestamp(post.created_utc).strftime('%Y-%m-%d %H:%M:%S')
    full_permalink = f"https://www.reddit.com{post.permalink}"
    
    # Gather post details
    post_data = {
        'post_title': post.title,
        'post_url': full_permalink,
        'score': post.score,
        'author': str(post.author),
        'created_date': created_date,
        'num_comments': post.num_comments,
        'selftext': post.selftext,
        'comments': []
    }
    
    # Fetch comments
    post.comments.replace_more(limit=0)  # This avoids loading additional comments
    for comment in post.comments.list():
        comment_data = {
            'comment_author': str(comment.author),
            'comment_body': comment.body,
            'comment_score': comment.score,
            'comment_created_date': datetime.utcfromtimestamp(comment.created_utc).strftime('%Y-%m-%d %H:%M:%S')
        }
        post_data['comments'].append(comment_data)
    
    parsed_posts.append(post_data)

# Save data to a JSON file
output_file_path = 'product_mentions_with_comments.json'
with open(output_file_path, 'w', encoding='utf-8') as json_file:
    json.dump(parsed_posts, json_file, ensure_ascii=False, indent=2)

print(f"Data has been saved to {output_file_path}")

/var/folders/4c/6f4zjqbs5tv0rl2p0nbskkp40000gn/T/ipykernel_32923/3055379407.py:21: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  created_date = datetime.utcfromtimestamp(post.created_utc).strftime('%Y-%m-%d %H:%M:%S')
/var/folders/4c/6f4zjqbs5tv0rl2p0nbskkp40000gn/T/ipykernel_32923/3055379407.py:43: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  'comment_created_date': datetime.utcfromtimestamp(comment.created_utc).strftime('%Y-%m-%d %H:%M:%S')


Data has been saved to product_mentions_with_comments.json


In [6]:
import json
import re
from collections import Counter
from typing import List, Dict
def load_comments(file_path: str) -> List[str]:
    """Load comments from a JSON file."""
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    comments = []
    for post in data:
        for comment in post.get('comments', []):
            comments.append(comment.get('comment_body', ''))
    return comments

def tokenize(text: str) -> List[str]:
    """Tokenize text into words."""
    text = text.lower()
    words = re.findall(r'\b[a-z]+\b', text)
    return words
def main():
    input_file_path = 'product_mentions_with_comments.json'
    comments = load_comments(input_file_path)
    all_words = []
    for comment in comments:
        all_words.extend(tokenize(comment))
    stopwords = set([
        'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves',
        'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their',
        'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was',
        'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and',
        'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between',
        'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on',
        'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all',
        'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same',
        'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'now'
    ])
    filtered_words = [word for word in all_words if word not in stopwords]
    word_counts = Counter(filtered_words)
    top_50_words = word_counts.most_common(200)
    output_data = [{'word': word, 'count': count} for word, count in top_50_words]
    output_file_path = 'top_words.json'
    with open(output_file_path, 'w', encoding='utf-8') as file:
        json.dump(output_data, file, ensure_ascii=False, indent=2)

    print(f"Top recurring words have been saved to {output_file_path}")

if __name__ == "__main__":
    main()

Top recurring words have been saved to top_words.json
